<a href="https://colab.research.google.com/github/Sh-Dulguun/Dulguun.github.io/blob/master/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preprocessings**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.datasets import mnist
from matplotlib import pyplot as plt

# **[Question  1] Looking back on the scratch**

In [ ]:
df = pd.read_csv("Iris.csv")
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)
y = np.array(y)
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# **[Problem 2] Consider the correspondence between Scratch and TensorFlow**

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Configuring Hyperparameters
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# Determine the shape of the arguments to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [ ]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# Reading the network structure
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# Optimization techniques
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated results
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# Index value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing a variable
init = tf.global_variables_initializer()


# Running Calculation Graphs
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop every epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop per mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))


print("It is much faster than implemented CNN from scratch. It's also very easy to use \
\nFirst, weights and biases are initialized and then layers are defined.")

Epoch 0, loss : 47.9514, val_loss : 284.0979, acc : 0.625
Epoch 1, loss : 43.0378, val_loss : 254.2579, acc : 0.625
Epoch 2, loss : 38.3406, val_loss : 225.3243, acc : 0.625
Epoch 3, loss : 33.7537, val_loss : 196.6855, acc : 0.625
Epoch 4, loss : 29.2011, val_loss : 167.4652, acc : 0.625
Epoch 5, loss : 24.3960, val_loss : 136.3264, acc : 0.625
Epoch 6, loss : 19.2588, val_loss : 103.7501, acc : 0.625
Epoch 7, loss : 14.0444, val_loss : 70.9598, acc : 0.625
Epoch 8, loss : 8.8916, val_loss : 38.7318, acc : 0.625
Epoch 9, loss : 4.1360, val_loss : 18.8356, acc : 0.125
Epoch 10, loss : 2.5106, val_loss : 30.8900, acc : 0.375
Epoch 11, loss : 2.6513, val_loss : 25.4411, acc : 0.375
Epoch 12, loss : 1.7809, val_loss : 9.8705, acc : 0.438
Epoch 13, loss : 1.3557, val_loss : 6.5804, acc : 0.438
Epoch 14, loss : 1.0566, val_loss : 7.7044, acc : 0.438
Epoch 15, loss : 0.9485, val_loss : 7.1685, acc : 0.438
Epoch 16, loss : 0.7923, val_loss : 4.1130, acc : 0.500
Epoch 17, loss : 0.6604, val_lo

# **[Problem 3] Create a model of Iris using all three types of objective variables**

In [ ]:
df = pd.read_csv("Iris.csv")
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica") | (df["Species"]=="Iris-setosa")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)
y = np.array(y)
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y[y == "Iris-setosa"] = 2

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:,np.newaxis])

X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]

n_classes = 3
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop every epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop per mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

<ipython-input-48-30d56e557bd2>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 1.3033, val_loss : 9.7414, acc : 0.403
Epoch 1, loss : 0.7933, val_loss : 5.0665, acc : 0.486
Epoch 2, loss : 0.4429, val_loss : 2.9194, acc : 0.611
Epoch 3, loss : 0.2443, val_loss : 1.3513, acc : 0.708
Epoch 4, loss : 0.0955, val_loss : 0.5434, acc : 0.875
Epoch 5, loss : 0.0501, val_loss : 0.5281, acc : 0.875
Epoch 6, loss : 0.0363, val_loss : 0.3961, acc : 0.917
Epoch 7, loss : 0.0279, val_loss : 0.3500, acc : 0.931
Epoch 8, loss : 0.0247, val_loss : 0.3298, acc : 0.917
Epoch 9, loss : 0.0223, val_loss : 0.3184, acc : 0.917
Epoch 10, loss : 0.0208, val_loss : 0.3011, acc : 0.917
Epoch 11, loss : 0.0195, val_loss : 0.2863, acc : 0.931
Epoch 12, loss : 0.0183, val_loss : 0.2731, acc : 0.931
Epoch 13, loss : 0.0172, val_loss : 0.2599, acc : 0.931
Epoch 14, loss : 0.0162, val_loss : 0.2479, acc : 0.931
Epoch 15, loss : 0.0153, val_loss : 0.2375, acc : 0.931
Epoch 16, loss : 0.0145, val_loss : 0.2282, acc : 0.931
Epoch 17, loss : 0.0137, val_loss : 0.2196, acc : 0.931
Ep

# **[Question 4] Create a House Prices model**

In [ ]:
df = pd.read_csv("train.csv")
X = df[['GrLivArea', 'YearBuilt']].to_numpy()
y = df[['SalePrice']].to_numpy()
print("Xshape:", X.shape)
print("yshape:", y.shape)
X = np.log1p(X)
y = np.log1p(y)

print("Xshape:", X.shape)
print("yshape:", y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

Xshape: (1460, 2)
yshape: (1460, 1)
Xshape: (1460, 2)
yshape: (1460, 1)


In [ ]:
learning_rate = 0.001
batch_size = 10
num_epochs = 50
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

logits = example_net(X)
loss_op =  tf.losses.mean_squared_error(labels=Y, predictions=logits)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop every epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop per mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

<ipython-input-48-30d56e557bd2>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 7.9568, val_loss : 2.2204, acc : 1.000
Epoch 1, loss : 0.1623, val_loss : 0.8613, acc : 1.000
Epoch 2, loss : 0.0791, val_loss : 0.4186, acc : 1.000
Epoch 3, loss : 0.0450, val_loss : 0.2205, acc : 1.000
Epoch 4, loss : 0.0275, val_loss : 0.1293, acc : 1.000
Epoch 5, loss : 0.0172, val_loss : 0.0875, acc : 1.000
Epoch 6, loss : 0.0130, val_loss : 0.0757, acc : 1.000
Epoch 7, loss : 0.0112, val_loss : 0.0690, acc : 1.000
Epoch 8, loss : 0.0102, val_loss : 0.0658, acc : 1.000
Epoch 9, loss : 0.0095, val_loss : 0.0639, acc : 1.000
Epoch 10, loss : 0.0089, val_loss : 0.0640, acc : 1.000
Epoch 11, loss : 0.0085, val_loss : 0.0634, acc : 1.000
Epoch 12, loss : 0.0082, val_loss : 0.0624, acc : 1.000
Epoch 13, loss : 0.0079, val_loss : 0.0614, acc : 1.000
Epoch 14, loss : 0.0077, val_loss : 0.0606, acc : 1.000
Epoch 15, loss : 0.0075, val_loss : 0.0602, acc : 1.000
Epoch 16, loss : 0.0073, val_loss : 0.0600, acc : 1.000
Epoch 17, loss : 0.0072, val_loss : 0.0601, acc : 1.000
Ep

# **[Problem 5] Create an MNIST model**

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

X_train /= 255
X_test /= 255

y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:])
y_test_one_hot = enc.fit_transform(y_test[:])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [ ]:
learning_rate = 0.001
batch_size = 10
num_epochs = 40
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn.softmax(logits),1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

<ipython-input-48-30d56e557bd2>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)


Epoch 0, loss : 2.9673, val_loss : 7.4110, acc : 0.754
Epoch 1, loss : 0.4277, val_loss : 2.5285, acc : 0.743
Epoch 2, loss : 0.1550, val_loss : 1.2647, acc : 0.758
Epoch 3, loss : 0.0922, val_loss : 0.9985, acc : 0.792
Epoch 4, loss : 0.0705, val_loss : 0.8861, acc : 0.823
Epoch 5, loss : 0.0589, val_loss : 0.8036, acc : 0.836
Epoch 6, loss : 0.0520, val_loss : 0.7735, acc : 0.851
Epoch 7, loss : 0.0464, val_loss : 0.6713, acc : 0.867
Epoch 8, loss : 0.0418, val_loss : 0.6287, acc : 0.881
Epoch 9, loss : 0.0379, val_loss : 0.6111, acc : 0.888
Epoch 10, loss : 0.0350, val_loss : 0.6090, acc : 0.887
Epoch 11, loss : 0.0331, val_loss : 0.5697, acc : 0.897
Epoch 12, loss : 0.0309, val_loss : 0.5718, acc : 0.897
Epoch 13, loss : 0.0294, val_loss : 0.5829, acc : 0.902
Epoch 14, loss : 0.0276, val_loss : 0.5394, acc : 0.905
Epoch 15, loss : 0.0266, val_loss : 0.5462, acc : 0.913
Epoch 16, loss : 0.0257, val_loss : 0.5250, acc : 0.909
Epoch 17, loss : 0.0239, val_loss : 0.5135, acc : 0.916
Ep